In [3]:
# For preventing some error message
import gym
gym.logger.set_level(40)

In [4]:
import os
import functools

import cv2
import numpy as np

from tests.metaworld.envs.mujoco.sawyer_xyz.test_scripted_policies import ALL_ENVS, test_cases_latest_nonoise

ImportError: cannot import name 'envs' from partially initialized module 'metaworld' (most likely due to a circular import) (/home/ykai/anaconda3/envs/Testaa/lib/python3.9/site-packages/metaworld/__init__.py)

In [32]:
!LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libGLEW.so:/usr/lib/nvidia/libGL.so


In [33]:
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/lib/nvidia
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/lib/nvidia-510
!export LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libGLEW.so

In [34]:
import os
os.getcwd()

'/ssd_2/ssd2048/papers/Sensory_extended_metaworld/metaworld/scripts'

In [35]:
def my_trajectory_generator(env, policy, act_noise_pct, res=(640, 480), camera='corner1', depth = True):
    action_space_ptp = env.action_space.high - env.action_space.low
    env.reset()
    env.reset_model()
    o = env.reset()

    for _ in range(env.max_path_length):
        a = policy.get_action(o)
        a = np.random.normal(a, act_noise_pct * action_space_ptp)

        o, r, done, info = env.step(a)
        # Camera is one of ['corner', 'topview', 'behindGripper', 'gripperPOV']
        # print(np.shape(env.sim.render(*res, mode='offscreen', camera_name=camera)[:,:,::-1]))
        img, d = env.sim.render(*res, mode='offscreen', camera_name=camera,depth = depth)
        # print(depth)
        yield r, done, info, img, d
        
def write_for_img(tag, img):
    if not os.path.exists('latentfusion_inputs'):
        os.mkdir('latentfusion_inputs')
    name = f'latentfusion_inputs/{tag}.png'
    return cv2.imwrite(name, img)

In [36]:
resolution = (640, 480)
camera = ['topview', 'corner1', 'corner2', 'corner3', 'behindGripper']
flip=True # if True, flips output image 180 degrees

config = [
    # env, action noise pct, cycles, quit on success
    ('assembly-v2', np.zeros(4), 1, True),
]
for camera in camera:
    if camera in ['corner1', 'corner2', 'corner3']:
        flip = False
    else:
        flip = True
    for env, noise, cycles, quit_on_success in config:
        tag = env + '-noise-' + np.array2string(noise, precision=2, separator=',', suppress_small=True)\
            + '-cycles-'+ str(cycles) +'-camera-'+ camera

        policy = functools.reduce(lambda a,b : a if a[0] == env else b, test_cases_latest_nonoise)[1]
        env = ALL_ENVS[env]()
        env._partially_observable = False
        env._freeze_rand_vec = False
        env._set_task_called = True
        for _ in range(cycles):
            for r, done, info, img, depth in my_trajectory_generator(env, policy, noise, resolution, camera, depth = True):
                if flip: img = cv2.rotate(img, cv2.ROTATE_180); depth = cv2.rotate(depth, cv2.ROTATE_180)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                write_for_img(tag, img)
                
                print(depth)
                depth = (np.max(depth)-depth) / (np.max(depth) - np.min(depth))
                # depth = (depth-np.min(depth)) / (np.max(depth) - np.min(depth))
                depth = np.asarray(depth * 15, dtype=np.uint8)
                print(np.shape(depth))
                print(depth)
                write_for_img(tag+'_depth', depth)
                
                break

[[0.9908411 0.9908411 0.9908411 ... 0.9908411 0.9908411 0.9908411]
 [0.9908411 0.9908411 0.9908411 ... 0.9908411 0.9908411 0.9908411]
 [0.9908411 0.9908411 0.9908411 ... 0.9908411 0.9908411 0.9908411]
 ...
 [0.9908411 0.9908411 0.9908411 ... 0.9908411 0.9908411 0.9908411]
 [0.9908411 0.9908411 0.9908411 ... 0.9908411 0.9908411 0.9908411]
 [0.9908411 0.9908411 0.9908411 ... 0.9908411 0.9908411 0.9908411]]
(480, 640)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


ValueError: No "camera" with name corner1 exists. Available "camera" names = ('topview', 'my_topview1', 'corner', 'corner2', 'corner3', 'my_corner3', 'behindGripper', 'gripperPOV').

In [38]:
import glob
import os
import sys

def get_egg_file(module_name):
    def f(packages):
        return glob.glob(
            os.path.join(os.path.dirname(os.path.dirname(sys.executable)),
                         'lib', 'python*', packages, module_name + '.egg-link'))

    return f('site-packages') or f('dist-packages')


egg_file = get_egg_file('metaworld')
print(egg_file)
# if egg_file:
#     os.remove(egg_file[0])


[]
